In [1]:
import polars as pl
import calendar
import gc
import json
import glob
import hashlib

In [2]:
def j2d(f):
    with open(f, "rt") as file:
        d = json.load(file)
    return d

def d2j(d, f):
    with open(f, "wt") as file:
        json.dump(d, file, indent = 2)

In [3]:
d_ss = j2d("../../share3/mat/d_ss.json")

In [4]:
d_ss

{'solt': '89sryf', 'stretch': 62}

In [5]:
l_dis = j2d("../../share3/mat/dm_codes.json")['dmcode']
# disset = set(l_dis)
l_dis[:3]

['8843375', '2500031', '8838633']

In [6]:
l_can = j2d("../../share3/mat/cans_codes.json")['cancercode']
l_can[:3]

['8845943', '8845270', '8844174']

In [7]:
disset = set(l_dis + l_can)

In [8]:
# f = "../../share3/temp/receipt_diseases.parquet"
f = "../../share3/receipt_diseases.parquet"
df_dis = pl.scan_parquet(f, low_memory=True, cache=False)\
    .select(
        pl.col("kojin_id"),
        pl.col("receipt_ym"),
        pl.col("receipt_id"),
        pl.col("line_no"),
        pl.col("diseases_code"),
        pl.col("utagai_flg")
    )\
    .with_columns(
        pl.col("kojin_id").hash(seed=d_ss["stretch"]).cast(pl.String).str.slice(-1,1).alias("grp"),
        # pl.col("kojin_id").str.slice(-2,2).alias("grp")
        # pl.col("kojin_id").map_elements(lambda s:s2h(s)).alias("grp")
    )

# df_dis.fetch()

In [9]:
df_dis_fix = df_dis\
    .filter(pl.col("utagai_flg") == "0")\
    .filter(pl.col("diseases_code").is_in(disset))

In [10]:
%%time
f_temp = "../../share3/df_dis_filtered.parquet"
# df_dis_fix.sink_parquet(f_temp,maintain_order=False)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.91 µs


In [16]:
del df_dis_fix
del df_dis

NameError: name 'df_dis_fix' is not defined

In [17]:
gc.collect()

953

In [11]:
%%time
# f = "../../share3/df_dis_split_filtered.parquet"
df_dis = pl.scan_parquet(f_temp, low_memory=True)

CPU times: user 3.04 s, sys: 1.03 s, total: 4.06 s
Wall time: 4.02 s


In [12]:
gc.collect()

374

In [13]:
%%time
lx = [str(i) for i in range(10)]
lx

CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 18.1 µs


['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [15]:
%%time

for k in lx:
    # print(k)
    f_out = "../../share3/dis/df_dis_split_{}.parquet".format(k)
    print(f_temp, f_out)
    df_dis\
        .filter(pl.col("grp") == k)\
        .sink_parquet(f_out,maintain_order=True)

    # del df_dis
    gc.collect()

../../share3/df_dis_filtered.parquet ../../share3/dis/df_dis_split_0.parquet
../../share3/df_dis_filtered.parquet ../../share3/dis/df_dis_split_1.parquet
../../share3/df_dis_filtered.parquet ../../share3/dis/df_dis_split_2.parquet
../../share3/df_dis_filtered.parquet ../../share3/dis/df_dis_split_3.parquet
../../share3/df_dis_filtered.parquet ../../share3/dis/df_dis_split_4.parquet
../../share3/df_dis_filtered.parquet ../../share3/dis/df_dis_split_5.parquet
../../share3/df_dis_filtered.parquet ../../share3/dis/df_dis_split_6.parquet
../../share3/df_dis_filtered.parquet ../../share3/dis/df_dis_split_7.parquet
../../share3/df_dis_filtered.parquet ../../share3/dis/df_dis_split_8.parquet
../../share3/df_dis_filtered.parquet ../../share3/dis/df_dis_split_9.parquet
CPU times: user 1h 1min 53s, sys: 2h 22min 23s, total: 3h 24min 17s
Wall time: 2h 54min 12s
